# Blue Brain BioExplorer

![](./images/bioexplorer_banner.png)

In [ ]:
from bioexplorer import BioExplorer
from braynsmediamaker import MovieMaker
import os

sequence_name = 'low_glucose'

hostname = 'localhost:5000'
resource_folder = '../tests/test_files/'
    
be = BioExplorer(hostname)
core = be.core_api()
mm = MovieMaker(core)
bio_explorer_version = be.version()
print(bio_explorer_version)
be.reset()

output_folder = resource_folder + '/movies/frames/' + \
    bio_explorer_version + '/' + sequence_name + '/stills/'
print(output_folder)

## Data loading

In [ ]:
cache_folder = resource_folder + 'caches/' + bio_explorer_version + '/'
print(cache_folder)

In [ ]:
cache_filename = cache_folder + sequence_name + '_scenario.bioexplorer'
print('Loading ' + cache_filename)
dummy = core.add_model(cache_filename)

In [ ]:
nb_models = len(core.scene.models)
if nb_models != 147:
    print('ERROR: Not all models have been loaded!!! Expected 147, found ' + str(nb_models))
else:
    print('All good!')

## Rendering settings

In [ ]:
status = core.set_renderer(
    background_color=[96 / 255, 125 / 255, 139 / 255],
    current='bio_explorer',
    samples_per_pixel=1, subsampling=1, max_accum_frames=1)
params = core.BioExplorerRendererParams()
params.gi_samples = 1
params.gi_weight = 0.4
params.gi_distance = 100
params.shadows = 1.0
params.soft_shadows = 1.0
params.soft_shadows_samples = 1
params.fog_start = 1300
params.fog_thickness = 1300
params.max_bounces = 1
status = core.set_renderer_params(params)

## Image settings

In [ ]:
image_size = [3840, 2160]
image_samples_per_pixel = 64

command_line='mkdir -p ' + output_folder
os.system(command_line)
command_line='ls ' + output_folder
print(output_folder)
if os.system(command_line) != 0:
    print('ERROR: Failed to create output folder')

## Snapshots

In [ ]:
cam = core.get_camera()
print('orientation=' + str(cam['orientation']) + ',')
print('position=' + str(cam['position']) + ',')
print('target=' + str(cam['target']))

In [ ]:
def set_all_materials_to_grey():
    for model in core.scene.models:
        model_id = model['id']
        model_name = model['name']

        material_ids = be.get_material_ids(model_id)['ids']
        nb_materials = len(material_ids)
        palette = list()
        for p in range(nb_materials):
            palette.append([0.25,0.25,0.25])
        be.set_materials(
            model_ids=[model_id], material_ids=material_ids, 
            diffuse_colors=palette, specular_colors=palette)

In [ ]:
def set_glycans_materials(high_mannose=False, complex=False, hybrid=False, o_glycans=False):
    glycans_colors = [[0, 1, 1], [1, 1, 0], [1, 0, 1], [0.2, 0.2, 0.7]]

    for model in core.scene.models:
        model_id = model['id']
        model_name = model['name']

        material_ids = be.get_material_ids(model_id)['ids']
        nb_materials = len(material_ids)
        if be.NAME_GLYCAN_HIGH_MANNOSE in model_name and high_mannose:
            palette = list()
            for m in range(nb_materials):
                palette.append(glycans_colors[0])
            be.set_materials(
                model_ids=[model_id], material_ids=material_ids,
                diffuse_colors=palette, specular_colors=palette
            )
        if be.NAME_GLYCAN_COMPLEX in model_name and complex:
            palette = list()
            for m in range(nb_materials):
                palette.append(glycans_colors[1])
            be.set_materials(
                model_ids=[model_id], material_ids=material_ids,
                diffuse_colors=palette, specular_colors=palette
            )            
        if be.NAME_GLYCAN_HYBRID in model_name and hybrid:
            palette = list()
            for m in range(nb_materials):
                palette.append(glycans_colors[2])
            be.set_materials(
                model_ids=[model_id], material_ids=material_ids,
                diffuse_colors=palette, specular_colors=palette
            )                
        if be.NAME_GLYCAN_O_GLYCAN in model_name and o_glycans:
            palette = list()
            for m in range(nb_materials):
                palette.append(glycans_colors[3])
            be.set_materials(
                model_ids=[model_id], material_ids=material_ids,
                diffuse_colors=palette, specular_colors=palette
            )

### Frame 600 - Spike and glycans

In [ ]:
status = core.set_camera(
    orientation=[-0.005685498736699895, 0.04744824242706568, -0.004076164405359884, 0.998849199968863],
    position=[201.58518981933594, 74.49932098388672, -106.41613006591797],
    target=[201.4903564453125, 74.48834991455078, -107.41156005859375]
)

In [ ]:
set_all_materials_to_grey()
mm.create_snapshot(exportIntermediateFrames=False,
    size=image_size, samples_per_pixel=image_samples_per_pixel,
    path=output_folder + '/00600.png')

In [ ]:
set_glycans_materials(complex=True)
mm.create_snapshot(exportIntermediateFrames=False,
    size=image_size, samples_per_pixel=image_samples_per_pixel,
    path=output_folder + '/00600a.png')

In [ ]:
set_glycans_materials(o_glycans=True)
mm.create_snapshot(exportIntermediateFrames=False,
    size=image_size, samples_per_pixel=image_samples_per_pixel,
    path=output_folder + '/00600b.png')

In [ ]:
set_glycans_materials(high_mannose=True)
mm.create_snapshot(exportIntermediateFrames=False,
    size=image_size, samples_per_pixel=image_samples_per_pixel,
    path=output_folder + '/00600c.png')

### Frame 1600 - Immune system

In [ ]:
status = core.set_camera(
    orientation=[-0.0036387755614786625, 0.24302943103787245, -0.002970781649413259, 0.9700075409079111],
    position=[238.1634521484375, 46.43696212768555, 372.584716796875],
    target=[237.69195556640625, 46.43134689331055, 371.702880859375]
)

In [ ]:
def apply_immune_colors():
    import seaborn as sns
    shading_mode = be.SHADING_MODE_BASIC
    specular_exponent = 50
    for model in core.scene.models:
        model_id = model['id']
        model_name = model['name']

        material_ids = be.get_material_ids(model_id)['ids']
        nb_materials = len(material_ids)

        if be.NAME_GLUCOSE in model_name:
            palette = sns.color_palette('Blues', nb_materials)
            be.set_materials_from_palette(
                shading_mode=shading_mode, specular_exponent=specular_exponent,
                model_ids=[model_id], material_ids=material_ids, palette=palette)

        if be.NAME_LACTOFERRIN in model_name:
            palette = sns.color_palette('afmhot', nb_materials)
            be.set_materials_from_palette(
                shading_mode=shading_mode, specular_exponent=specular_exponent,
                model_ids=[model_id], material_ids=material_ids, palette=palette)

        if be.NAME_DEFENSIN in model_name:
            palette = sns.color_palette('plasma_r', nb_materials)
            be.set_materials_from_palette(
                shading_mode=shading_mode, specular_exponent=specular_exponent,
                model_ids=[model_id], material_ids=material_ids, palette=palette)

        if be.NAME_SURFACTANT_HEAD in model_name:
            palette = sns.color_palette('OrRd_r', nb_materials)
            emission = 0
            be.set_materials_from_palette(
                shading_mode=shading_mode, specular_exponent=specular_exponent,
                model_ids=[model_id], material_ids=material_ids,
                palette=palette, emission=emission)
        if be.NAME_COLLAGEN in model_name:
            material_ids = list(be.get_material_ids(model_id)['ids'])
            nb_materials = len(material_ids)
            palette = list()
            emissions = list()
            for i in range(nb_materials):
                palette.append([1,1,1])
                emissions.append(0.1)
            be.set_materials(
                model_ids=[model_id], material_ids=material_ids,
                diffuse_colors=palette, specular_colors=palette,
                emissions=emissions
            )

In [ ]:
set_all_materials_to_grey()
apply_immune_colors()

In [ ]:
mm.create_snapshot(exportIntermediateFrames=False,
    size=image_size, samples_per_pixel=image_samples_per_pixel,
    path=output_folder + '/01600.png')

### Frame 2600 - Surfactant head

In [ ]:
status = core.set_camera(
    orientation=[0.08610159341705904, 0.475212226477359, -0.055074495502621494, 0.8739145583880651],
    position=[-9.827075958251953, 110.72057342529297, 60.944026947021484],
    target=[-10.648181915283203, 110.92340850830078, 60.41050720214844]
)

In [ ]:
mm.create_snapshot(exportIntermediateFrames=False,
    size=image_size, samples_per_pixel=image_samples_per_pixel,
    path=output_folder + '/02600.png')